In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tourism_with_id.csv to tourism_with_id (1).csv
User uploaded file "tourism_with_id.csv" with length 365796 bytes


In [ ]:
import pandas as pd

df = pd.read_csv('tourism_with_id.csv')  # replace 'yourfile.csv' with your uploaded file name

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Load data
df = pd.read_csv('tourism_with_id.csv')

# Preprocessing the descriptions
df['Description'] = df['Description'].apply(lambda x: x.lower().split())

# Training a Word2Vec model
model = Word2Vec(df['Description'], min_count=1)

def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
    list_of_docs : list of documents
    model : Word2Vec model

    Returns: 
    List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = [model.wv[token] for token in tokens if token in model.wv.key_to_index]
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

# Vectorize the descriptions
description_vectors = vectorize(df['Description'].values, model)

def recommend_destinations(user_input, city=None):
    # Filter destinations based on city if city is not None
    if city is not None:
        city_df = df[df['City'] == city]
    else:
        city_df = df

    # Transform the user input into the same vector space
    user_input_vector = vectorize([user_input.lower().split()], model)

    # Calculate the cosine similarity between the user input vector and the description_vectors
    city_description_vectors = vectorize(city_df['Description'].values, model)
    cosine_sim = cosine_similarity(user_input_vector, city_description_vectors)

    # Get the indices of the destinations sorted by their similarity to the user input
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 3 most similar destinations
    destination_indices = [i[0] for i in sim_scores[:3]]

    # Return the top 3 most similar destinations
    return city_df['Place_Name'].iloc[destination_indices]
# Test the function with a user input
print(recommend_destinations("pantai bagus", city='Yogyakarta'))



167      Pantai Timang
196     Pantai Jungwok
201    Pantai Ngandong
Name: Place_Name, dtype: object
